In [84]:
import nltk
import string
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pprint import pprint

In [152]:
import gensim
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt

# Perform LDA using Gensim package

In [86]:
punc1 = '!"#$%\'()*.:;<=>?@[\\]^`{|}~’“”‘–-' # adapted string.punctuation
punc2 = ['=', '/', '&', '_', '+', '…', '...']

In [176]:
# Create stopwords using a systematic approach
stopWords = stopwords.words('english')
stopWords = set([word.replace("'", "") for word in stopWords])
singleLetters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
for letter in singleLetters:
    stopWords.add(letter)
legend = set(["pt", "md", "oth", "so", "legend", "inaudible", "phi", "laughs", "pt/so", "mdmd", "patient", "physician", "clean"])
stopWords = stopWords.union(legend)
filler = set(["um", "hmm", "uh", "mm", "uhhuh", "uhum", "mmmhmm", "uhmmm", "nah", "whatnot", "mhmmm", "uhhmm", "othumhmm", "mmhmm", "umhmm","oh", "ah", "hm","ok", "okay", "kay", "umm","gee", "yeah","yep", "jeez", "huh", "ya", "mmhmm", "mmm", "hum", "cause", "alot", "kinda", "stuff", "like","right", "yup", "hi", "nope", "hey", "cuz", "etc", "mmhm", "mhm", "ahh", "hello", "gosh", "bye", "uhh", "er", "yea", "geez", "ohh", "heh", "ahhh", "ohhh", "aah", "yada", "whoa", "aaah", "okey", "dokey", "uhmhmmm", "nahuh", "ahhm", "yeyeah", "uhhhhh", "uhhm", "uhm", "hmmm", "eh", "ha"])
stopWords = stopWords.union(filler)
pr = set(["hell", "shell","thereof", "mustve","wheres", "hows", "hed", "theyve", "wanna", "goin", "got", "get","would", "well", "im", "itd", "theyll","go", "theres", "em","thats","could", "aint","gonna", "ill", "theyre", "ive", "us", "cant", "id", "lets", "hes", "wed", "weve", "came", "sounds", "whats", "hes", "thered","whatd", "doin", "mightve", "gal", "whos", "itll", "'em"])
stopWords = stopWords.union(pr)
names = set(["henry", "walsh", "potter"])
stopWords = stopWords.union(names)
misspell = set(["anand", "andand", "thethe", "ifif", "itit", "ii", "thatsthat", "eek", "uhoh", "wewe", "isis", "nahuh", "th", "'", "'cause", "youl"])
stopWords = stopWords.union(misspell)
shortwords = set(['nn', 'rd', 'st', 'de', 'yer', 'le', 'ti', 'le', 'nd', 'na', 'da'])
stopWords = stopWords.union(shortwords)

In [90]:
# Functions to help with processing text
def remove_parentheses(txt):
    txt = re.sub('\([^)]*\)\)','', txt) # remove double parentheses 
    txt = re.sub(r'\([^)]*\)', '', txt) # remove single parentheses 
    return txt

# def remove_numerical(txt):
#     txt = re.sub('[0-9]+', '', txt)
#     return txt

def remove_punc(txt):
    for a in punc2:
        txt = txt.replace(a," ")
    for b in punc1:
        txt = txt.replace(b,"")
    return txt

def lowercase(txt):
    txt = txt.lower()
    return txt

<input>:3: DeprecationWarning: invalid escape sequence \(
<input>:3: DeprecationWarning: invalid escape sequence \(
<input>:3: DeprecationWarning: invalid escape sequence \(
<ipython-input-90-246eac84cd8f>:3: DeprecationWarning: invalid escape sequence \(
  txt = re.sub('\([^)]*\)\)','', txt) # remove double parentheses


In [126]:
# Functions to help with tokenizing, cleaning up text
def sent_to_words(document):
    return gensim.utils.simple_preprocess(str(document), deacc=True) # deacc=True removes punctuations

def remove_stopwords(document, word_list):
    return [word for word in document if word not in word_list]

def lemma_tokens(tokens, lemmatizer = WordNetLemmatizer()):
    lemmed = []
    for item in tokens:
        lemmed.append(lemmatizer.lemmatize(item))
    return lemmed

# Process Texts

In [162]:
df = pd.read_csv('all_transcripts.csv')
df.head()

,Unnamed: 0,patient_id,Convo_1,Convo_2,Dataset,Doctor_1,Doctor_2
0,0,A003,A003 LEGEND: MD2=Physician PT=Patient MD2: So ...,NaN,VA,U,NaN
1,1,A004,A004 LEGEND: MD2=Physician OTH=Study staff-set...,NaN,VA,U,NaN
2,2,A014,A014 LEGEND: MD2=Physician PT=Patient MD2: Tha...,NaN,VA,U,NaN
3,3,A015,A015_CLEAN LEGEND: MD2=Physician PT=Patient MD...,NaN,VA,U,NaN
4,4,A016,A016_CLEAN_LOUD LEGEND: MD2=Physician PT: Pati...,NaN,VA,U,NaN


In [163]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [164]:
df_copy = df # in case we need to revisit later on, we have a copy

In [165]:
# we are cleaning up each of the conversations in Convo_1 column and then tokenizing the texts
conversations_tokenized = [] 
for index, row in df.iterrows():
    if pd.isnull(row['Convo_1']) == False:
        # remove parentheses, numbers, punctuation, and convert everything to lowercase
        row['Convo_1'] = remove_parentheses(row['Convo_1'])
        #row['Convo_1'] = remove_numerical(row['Convo_1'])
        row['Convo_1'] = remove_punc(row['Convo_1'])
        row['Convo_1'] = lowercase(row['Convo_1'])
        df.set_value(index,'Convo_1', row['Convo_1'])
        conversations_tokenized.append(sent_to_words(row['Convo_1'])) # create new column of processed, tokenized doucments

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [166]:
df.head()

,patient_id,Convo_1,Convo_2,Dataset,Doctor_1,Doctor_2
0,A003,a003 legend md2 physician pt patient md2 so th...,NaN,VA,U,NaN
1,A004,a004 legend md2 physician oth study staffsetti...,NaN,VA,U,NaN
2,A014,a014 legend md2 physician pt patient md2 thank...,NaN,VA,U,NaN
3,A015,a015 clean legend md2 physician pt patient md2...,NaN,VA,U,NaN
4,A016,a016 clean loud legend md2 physician pt patien...,NaN,VA,U,NaN


In [167]:
# remove stopwords
conversations_nostops = []
for convo in conversations_tokenized:
    convo_nostop = remove_stopwords(convo, stopWords)
    conversations_nostops.append(convo_nostop)

In [168]:
# lemmatize the words
conversations_lemmatized = []
for doc in conversations_nostops:
    convo_lemmtized = lemma_tokens(doc)
    conversations_lemmatized.append(convo_lemmtized)

# Create Bag of Words Model

In [169]:
# Create Dictionary
id2word = corpora.Dictionary(conversations_lemmatized)

In [170]:
# Filter Dictionary based on the following criteria
id2word.filter_extremes(no_below=3, no_above = 0.75, keep_n=7000, keep_tokens=None) 

In [171]:
# Create BOW Model
corpus = [id2word.doc2bow(text) for text in conversations_lemmatized]
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [181]:
len(id2word)

5687

In [149]:
# get term frequencies (exploring purposes)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abnormal', 2),
  ('active', 5),
  ('ahead', 1),
  ('almost', 1),
  ('anymore', 1),
  ('anyone', 1),
  ('appear', 1),
  ('appearance', 3),
  ('appears', 1),
  ('appointment', 1),
  ('area', 1),
  ('away', 1),
  ('based', 2),
  ('basically', 1),
  ('beam', 1),
  ('best', 3),
  ('bone', 2),
  ('bring', 2),
  ('called', 1),
  ('category', 4),
  ('cell', 2),
  ('center', 1),
  ('certainly', 1),
  ('change', 1),
  ('check', 2),
  ('clinic', 1),
  ('coming', 1),
  ('common', 4),
  ('computer', 1),
  ('consider', 1),
  ('control', 1),
  ('core', 5),
  ('cure', 2),
  ('delay', 1),
  ('designed', 1),
  ('diabetes', 1),
  ('dr', 1),
  ('dying', 1),
  ('educational', 1),
  ('eight', 1),
  ('enough', 5),
  ('erectile', 1),
  ('exam', 1),
  ('expected', 1),
  ('external', 1),
  ('fact', 1),
  ('felt', 1),
  ('fifteen', 1),
  ('five', 7),
  ('fix', 1),
  ('focus', 1),
  ('follow', 1),
  ('form', 1),
  ('four', 7),
  ('function', 2),
  ('general', 1),
  ('given', 1),
  ('gland', 3),
  ('grade', 4)

# Create LDA Model

In [177]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=2,
                                           chunksize=25,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [173]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.009*"instance" + 0.007*"clot" + 0.005*"florida" + 0.005*"hip" + '
  '0.005*"sheet" + 0.004*"million" + 0.003*"suck" + 0.003*"ground" + '
  '0.003*"replacement" + 0.003*"file"'),
 (1,
  '0.010*"gland" + 0.006*"tissue" + 0.006*"external" + 0.005*"pattern" + '
  '0.005*"hernia" + 0.005*"night" + 0.005*"area" + 0.005*"mri" + 0.005*"eye" + '
  '0.005*"lowrisk"'),
 (2,
  '0.008*"spot" + 0.007*"ultrasound" + 0.007*"length" + 0.006*"partner" + '
  '0.006*"cyberknife" + 0.006*"invasive" + 0.006*"slightly" + 0.005*"scrotum" '
  '+ 0.005*"whenever" + 0.005*"huge"'),
 (3,
  '0.003*"herpes" + 0.000*"downtime" + 0.000*"lurking" + 0.000*"ooooh" + '
  '0.000*"fishing" + 0.000*"insulin" + 0.000*"ptparticipant" + 0.000*"cane" + '
  '0.000*"heartbeat" + 0.000*"imminently"'),
 (4,
  '0.007*"toward" + 0.007*"colonoscopy" + 0.007*"nurse" + 0.006*"kid" + '
  '0.006*"dos" + 0.006*"bottom" + 0.005*"glass" + 0.005*"penile" + '
  '0.005*"assuming" + 0.004*"building"'),
 (5,
  '0.007*"lowgrade" + 0.007*

# Topic Coherence

In [174]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=conversations_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.46149909342716805


Look at how the various parameters impact coherence score (see how each parameter impacts in isolation versus more of a grid-search approach)

Brief non-technical explanation of topic coherence: https://www.quora.com/What-is-topic-coherence

### No_below
- Attempt 1: Coherence Score = 0.3878
- Attempt 2: no_below = 2, Coherence Score = 0.4577
- Attempt 3: no_below = 3, Coherence Score = 0.4577
- Attempt 4: no_below = 5, Coherence Score = 0.3878
- no_above = 0.8, keep_n = 5000, chunk_size = 50, update_every = 1, passes = 10

### No_above
- Attempt 1: Coherence Score = 0.45206915803222947
- Attempt 2: no_above = 0.9, Coherence Score = 0.4372
- Attempt 3: no_above = 0.8, Coherence Score = 0.4577
- Attempt 4: no_above = 0.75, Coherence Score = 0.4611
- Attempt 5: no_above = 0.70, Coherence Score = 0.4206
- no_below = 3, keep_n = 5000, chunk_size = 50, update_every = 1, passes = 10

### Keep_n
- Attempt 1: keep_n = 3000, Coherence Score:  0.4555
- Attempt 2: keep_n = 5000, Coherence Score:  0.4611
- Attempt 3: keep_n = 7000, Coherence Score:  0.4747
- no_below = 3, no_above = 0.75, chunk_size = 50, update_every = 1, passes = 10

### Num_passes
- Attempt 1: Passes = 5, Coherence Score = 0.4347
- Attempt 2: Passes = 10, Coherence Score = 0.4819
- Attempt 3: Passes = 20, Coherence Score = 0.4965
- no_below = 3, no_above = 0.75, keep_n = 7000, chunk_size = 50, update_every = 1

- Attempt 1: Passes = 5, Coherence Score = 0.4627
- Attempt 2: Passes = 10, Coherence Score = 0.4747
- Attempt 3: Passes = 20, Coherence Score = 0.5073
- no_below = 3, no_above = 0.75, keep_n = 7000, chunk_size = 25, update_every = 2

### Num_topics
- Attempt 1: num_topics = 5, Coherence Score = 0.3472
- Attempt 2: num_topics = 10, Coherence Score = 0.4747
- Attempt 3: num_topics = 15, Coherence Score = 0.3778
- no_below = 3, no_above = 0.75, keep_n = 7000, chunk_size = 25, update_every = 2, passes = 10

# Visualizing Topics

In [175]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.173422  0.111542       1        1  77.350525
1      0.219459  0.095535       2        1   9.285651
6      0.010083 -0.172081       3        1   4.451551
8     -0.003476 -0.022416       4        1   2.337774
2     -0.003331 -0.017317       5        1   1.866717
13    -0.004140 -0.012995       6        1   1.598609
4     -0.005432 -0.006059       7        1   1.113941
10    -0.005589 -0.002039       8        1   0.727195
5     -0.005253  0.000924       9        1   0.454636
11    -0.005324  0.001668      10        1   0.381747
0     -0.005448  0.002016      11        1   0.355530
12    -0.004622  0.004844      12        1   0.063694
3     -0.004536  0.005392      13        1   0.006626
7     -0.004484  0.005493      14        1   0.003434
14    -0.004484  0.005493      15        1   0.002372, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
36    Default  880.000000             dr  880.000000  30.0000  30.0000
334   Default  601.000000            men  601.000000  29.0000  29.0000
57    Default  332.000000          gland  332.000000  28.0000  28.0000
127   Default  576.000000         sexual  576.000000  27.0000  27.0000
1571  Default  498.000000           seed  498.000000  26.0000  26.0000
141   Default  459.000000   surveillance  459.000000  25.0000  25.0000
1250  Default  453.000000       approach  453.000000  24.0000  24.0000
634   Default  445.000000        percent  445.000000  23.0000  23.0000
131   Default  427.000000            six  427.000000  22.0000  22.0000
321   Default  426.000000     likelihood  426.000000  21.0000  21.0000
120   Default  418.000000         sample  418.000000  20.0000  20.0000
54    Default  416.000000       function  416.000000  19.0000  19.0000
366   Default  413.000000        patient  413.000000  18.0000  18.0000
346   Default  407.000000          nerve  407.000000  17.0000  17.0000
148   Default  405.000000          three  405.000000  16.0000  16.0000
920   Default  400.000000           scan  400.000000  15.0000  15.0000
917   Default  398.000000        robotic  398.000000  14.0000  14.0000
30    Default  389.000000        control  389.000000  13.0000  13.0000
156   Default  381.000000      typically  381.000000  12.0000  12.0000
157   Default  374.000000        urinary  374.000000  11.0000  11.0000
20    Default  364.000000           cell  364.000000  10.0000  10.0000
1765  Default  363.000000           dose  363.000000   9.0000   9.0000
247   Default  352.000000            dry  352.000000   8.0000   8.0000
1144  Default  352.000000        hormone  352.000000   7.0000   7.0000
58    Default  345.000000          grade  345.000000   6.0000   6.0000
31    Default  340.000000           core  340.000000   5.0000   5.0000
437   Default  338.000000           sort  338.000000   4.0000   4.0000
13    Default  335.000000      basically  335.000000   3.0000   3.0000
22    Default  332.000000      certainly  332.000000   2.0000   2.0000
778   Default  331.000000           data  331.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
5686  Topic15    0.001475       withwith   32.127293   0.6607  -8.6459
1865  Topic15    0.001475           tool   55.408993   0.1157  -8.6459
1889  Topic15    0.001475   circumstance   88.425262  -0.3517  -8.6459
1890  Topic15    0.001475       classify   12.007483   1.6449  -8.6459
1891  Topic15    0.001475       compared  132.670105  -0.7575  -8.6459
1892  Topic15    0.001475           cook   12.412905   1.6117  -8.6459
1893  Topic15    0.001475          creep   13.243487   1.5469  -8.6459
1894  Topic15    0.001475        crystal   13.294204   1.5431  -8.6459
1895  Topic15    0.001475          delve   12.171314   1.6313  -8.6459
1896  Topic15    0.001475     estimation   11.592242   1.

In this plot, we would want non-intersecting topics (bubbles) in different coordinates. The size of the bubble represents the prevalence of the topic, and we would expect some to be larger than others (although we want them to be generally large because that means that each topic is important, versus having many small bubbles). However, we mostly see overlapping topics of the same size in our topic models, which means that the topic model did not identify very distinct topics.

# Troubleshooting Word Processing

In [ ]:
for index, row in df.iterrows():
    for word in ['yer']: # de
        if pd.isnull(row['Convo_1']) == False and word in gensim.utils.simple_preprocess(str(row['Convo_1']), deacc=True):
            print(row)